In [12]:
import glob
import os
import fuse, causal_utils
from kl_general import sharpness_correction
glob.glob("/ist/ist-share/scads/can/robust_nlu/korn_reweight/nli/*/")

['/ist/ist-share/scads/can/robust_nlu/korn_reweight/nli/outputs_bert_base_overlapping_korn_ps3class_3/',
 '/ist/ist-share/scads/can/robust_nlu/korn_reweight/nli/outputs_bert_base_overlapping_korn_ps3class_2/',
 '/ist/ist-share/scads/can/robust_nlu/korn_reweight/nli/outputs_bert_base_overlapping_korn_ps3class_1/']

In [13]:
!ls /raid/can/nli_models/baseline_mind_distill/nli/seed1/

best.th		      model_state_e2_b0.th  raw_train.jsonl
config.json	      model_state_e3_b0.th  temperature.pt
meta.json	      model.tar.gz	    training_state_e2_b0.th
metrics_epoch_0.json  normal		    training_state_e3_b0.th
metrics_epoch_1.json  out.log		    vocabulary
metrics_epoch_2.json  raw_m.jsonl
metrics.json	      raw_mm.jsonl


In [67]:
!ls /raid/can/nli_models/baseline/nli/outputs_bert_base_clark_1_seed13370

outputs_bert_base_clark_1_seed13370  outputs_bert_base_clark_1_seed33370
outputs_bert_base_clark_1_seed23370


In [68]:
import pandas as pd
import numpy as np
from scipy.special import softmax
from scipy.special import expit

# select model
model_path='/raid/can/nli_models/baseline/nli/outputs_bert_base_clark_1_seed13370/'
# select data path
data_path='/ist/users/canu/debias_nlu/data/nli/'
df = pd.read_json(model_path+'raw_train.jsonl', lines=True)
# select fusion method here
fusion = fuse.sum_fuse

# Get avg  prob of bert

In [69]:
list_probs = []
for i in df['probs']:
    list_probs.extend(i)
x=np.array(list_probs)
avg=np.average(x,axis=0)

In [70]:
avg

array([0.33131394, 0.33371928, 0.33496678])

In [72]:
result_path

'/ist/ist-share/scads/can/robust_nlu/korn_reweight/nli/outputs_bert_base_overlapping_korn_ps3class_1/normal/'

# get score on dev set for correction

In [141]:
df_bias_dev = pd.read_json(
    data_path+'dev_prob_korn_lr_overlapping_sample_weight_3class.jsonl', lines=True)
bias_dev_score = [b for b in df_bias_dev['bias_probs']]
bias_dev_score = np.array(bias_dev_score)
y1m0_dev = fusion(avg, bias_dev_score)
df_bert_dev = pd.read_json(model_path+'raw_m.jsonl', lines=True)
y1m1prob_dev = []
for p, h in zip(df_bert_dev['probs'], y1m0_dev):
    new_y1m1 = fusion(np.array(p), h)
    y1m1prob_dev.append(new_y1m1)
c = sharpness_correction(bias_dev_score, y1m1prob_dev) 

#  get score from the bias model

In [143]:
# avg prob
#s score from bias model
df_hans = pd.read_json(data_path+'hans_prob_korn_lr_overlapping_sample_weight_3class.jsonl', lines=True)
hans_score=[b for b in df_hans['bias_probs'] ]
hans_score=np.array(hans_score)

# y1m0

In [160]:
bias_score=fusion(avg,hans_score)
#y1m0

In [145]:
bias_score[0]

array([-0.29660777, -0.47052674, -0.50219184])

In [146]:
result_path=model_path+'normal/'
# bert model predictions on HANS
df_bert = pd.read_json(result_path+'hans_result.jsonl', lines=True)

In [147]:
# ent = []
y1m1prob = []
for p,h in zip(df_bert['probs'],hans_score):
    new_y1m1 = fusion(np.array(p),h)
    y1m1prob.append(new_y1m1)
  

In [148]:
softmax

<function scipy.special._logsumexp.softmax(x, axis=None)>

In [149]:
c

array([0.44338705, 0.47912954, 0.45134683])

# corrected y1m0

In [150]:
# c = sharpness_correction(hans_score, y1m1prob) 

bias_score = fusion(c,hans_score)

In [151]:
c

array([0.44338705, 0.47912954, 0.45134683])

In [152]:
y1m1prob

[array([-0.35005494, -0.3039916 , -0.64651254]),
 array([-0.39038517, -0.27162996, -0.64606746]),
 array([-0.30526682, -0.35204881, -0.64101044]),
 array([-0.38887878, -0.27399987, -0.64356888]),
 array([-0.38837663, -0.27299918, -0.64629774]),
 array([-0.29790491, -0.35798832, -0.64499541]),
 array([-0.18062056, -0.56343552, -0.64367468]),
 array([-0.38654492, -0.2753041 , -0.64443831]),
 array([-0.33690861, -0.31671184, -0.64534499]),
 array([-0.3893537 , -0.27177804, -0.64729252]),
 array([-0.38209547, -0.27815953, -0.6454519 ]),
 array([-0.36818811, -0.28879663, -0.64610202]),
 array([-0.38844703, -0.27271522, -0.64675981]),
 array([-0.37598873, -0.28249437, -0.64622493]),
 array([-0.37159301, -0.28827739, -0.64185693]),
 array([-0.34048244, -0.315014  , -0.64241546]),
 array([-0.39022855, -0.27129145, -0.64697617]),
 array([-0.39110896, -0.2705109 , -0.64724623]),
 array([-0.38783593, -0.2729623 , -0.64717182]),
 array([-0.38983379, -0.27187561, -0.64638858]),
 array([-0.39175129,

In [153]:
bias_score

array([[-0.26901882, -0.41839738, -0.45784942],
       [-0.26901882, -0.41839738, -0.45784942],
       [-0.26901882, -0.41839738, -0.45784942],
       ...,
       [-0.26901882, -0.41839738, -0.45784942],
       [-0.26901882, -0.41839738, -0.45784942],
       [-0.26901882, -0.41839738, -0.45784942]])

# TIE

In [161]:
debias_scores = []
for p,b in zip(y1m1prob,bias_score):
    debias_scores.append(p-b) # TIE  

# Eval HANS

In [162]:
key = {0:"entailment",1:"contradiction",2:"neutral"}
labels = []
for i in debias_scores:
    labels.append(key[np.argmax(i)])
df_bert['debias_label']=labels

In [163]:
text_ans=""
for idx, obj in enumerate(df_bert['label']):
    text_ans = text_ans + "ex"+str(idx)+","+obj+"\n"   
    
text_ans_debias=""
for idx, obj in enumerate(df_bert['debias_label']):
    text_ans_debias = text_ans_debias + "ex"+str(idx)+","+obj+"\n"          

In [164]:
def format_label(label):
    if label == "entailment":
        return "entailment"
    else:
        return "non-entailment"

guess_dict = {}
for line in text_ans.split("\n"):
    if len(line)>1:
        parts = line.strip().split(",")
        guess_dict[parts[0]] = format_label(parts[1])
        
guess_dict_debias = {}
for line in text_ans_debias.split("\n"):
    if len(line)>1:
        parts = line.strip().split(",")
        guess_dict_debias[parts[0]] = format_label(parts[1])        

In [165]:
labels,baseline_avg=causal_utils.get_heur(guess_dict) # normal

Heuristic entailed results:
lexical_overlap: 0.9896
subsequence: 0.9994
constituent: 0.9998

Heuristic non-entailed results:
lexical_overlap: 0.6146
subsequence: 0.056
constituent: 0.0526
avg: 0.6186666666666666


In [166]:
labels,debias_avg=causal_utils.get_heur(guess_dict_debias) # TIE

Heuristic entailed results:
lexical_overlap: 0.9886
subsequence: 0.9994
constituent: 0.9998

Heuristic non-entailed results:
lexical_overlap: 0.6254
subsequence: 0.0592
constituent: 0.0574
avg: 0.6216333333333334


# Causal Mediation Analysis

In [115]:
def get_ans(ans):
    if ans == 0:
        return 'entailment'
    else:
        return 'non-entailment' 
    
# y0m0
import pickle
modelname='mnli_lr_model.sav'
loaded_model = pickle.load(open(modelname, 'rb'))
x0=loaded_model.predict_proba(np.array([[0,0,0.41997876976119086]]))
m0=avg
y0m0=fusion(x0,m0)    

In [123]:
factual_pred_correct = []
TIE_pred_correct = []
NIE_pred_correct = []
INTmed_pred_correct = []
pred_correct = []
all_TIE = []
all_NIE = []
all_NDE = []
all_INTmed = []
for i in range(len(labels)): 
    y1m1 = y1m1prob[i]
    y1m0 = bias_score[i]
    TE = y1m1 - y0m0
    NDE = bias_score[i] - y0m0
    y0m1= fusion(x0,np.array(df_bert['probs'][i]) )
    TIE = y1m1 - y1m0
    NIE = y0m1 - y0m0
    INTmed = TIE - NIE
    # factual
    factual_ans = np.argmax(df_bert['probs'][i])
    factual_ans = get_ans(factual_ans)
    factual_correct = factual_ans==labels[i]   
    factual_pred_correct.append(factual_correct)
    # TIE
    TIE_ans = np.argmax(TIE)
    TIE_ans = get_ans(TIE_ans)
    TIE_correct = TIE_ans==labels[i]  
    TIE_pred_correct.append(TIE_correct)
    # INTmed
    INTmed_ans = np.argmax(INTmed[0])
    INTmed_ans = get_ans(INTmed_ans)
    INTmed_correct = INTmed_ans==labels[i]  
    INTmed_pred_correct.append(INTmed_correct)
    # NIE
    NIE_ans = np.argmax(NIE[0])
    NIE_ans = get_ans(NIE_ans)
    NIE_correct = NIE_ans==labels[i]  
    NIE_pred_correct.append(NIE_correct)    
    
    # save
    all_NDE.append(NDE[0][0])
    all_NIE.append(NIE[0][0])
    all_TIE.append(TIE[0])
    all_INTmed.append((INTmed[0][0]))
    if  (TIE[0]/TE[0][0])<9999999:
        cf_ans = np.argmax(np.array(df_bert['probs'][i]-hans_score[i]))
        cf_ans = get_ans(cf_ans)
        cf_correct = cf_ans==labels[i]
    else:
#         print(cf_ans)
        cf_correct = factual_ans ==labels[i]
    pred_correct.append(cf_correct)
    
#     np.array(df_bert['probs'][i]-bias_score)
#     labels[i]
print(np.array(all_TIE).mean(),(np.array(all_TIE).std()))
print(np.array(all_INTmed).mean(),(np.array(all_INTmed).std()))
print(np.array(all_NIE).mean(),(np.array(all_NIE).std()))

0.05897773440853179 0.04002435525754341
-0.02051590710053735 0.013665625425377402
0.07949364150906912 0.05368544027567681


In [119]:
np.array(df_bert['probs'][i]-hans_score[i])

array([-0.06199987, -0.03410642,  0.09610631])

In [120]:
df_bert['probs'][i]

[0.6700639724731441, 0.141605570912361, 0.18833048641681602]

In [121]:
hans_score[i]

array([0.73206384, 0.17571199, 0.09222417])

In [50]:
print(sum(NIE_pred_correct)/30000,sum(INTmed_pred_correct)/30000)

0.6439333333333334 0.4606


In [51]:
print(sum(factual_pred_correct)/30000,sum(pred_correct)/30000,sum(TIE_pred_correct)/30000)

0.6598666666666667 0.5182 0.6515


In [24]:
#TIE # 0.032416620996870094 # utama best seed  0.04260506151060745
#TIE # 0.0879242620437139 # PoE
#TIE # 0.10864416858494 # Baseline 0.10295758381662952
#
# baseline
# reweight_korn
# poe_korn


In [52]:
!ls /raid/can/nli_models/

baseline	       check_baseline  poe   poe3
baseline_mind_distill  korn_reweight   poe2  reweight_utama_github


In [53]:
!python cma.py

In [8]:
import cma
import glob
import numpy as np
import os
import fuse, causal_utils
model_path='/raid/can/nli_models/baseline/'
task='nli'
data_path='/ist/users/canu/debias_nlu/data/' + task + '/'
fusion = fuse.sum_fuse
test_set='hans'
correction=True

In [9]:
cma.report_CMA(model_path,task,data_path,test_set,fusion,correction)

/raid/can/nli_models/baseline/nli/outputs_bert_base_clark_1_seed33370/normal/
0.06294446807694988 0.07259367040949645
-0.023977854336051378 0.02528866896020864
0.08692232241300125 0.09787408550005666
/raid/can/nli_models/baseline/nli/outputs_bert_base_clark_1_seed13370/normal/
0.07404294678696302 0.06343940052637885
-0.027886371556130263 0.022113234206897377
0.1019293183430933 0.08554443506460487
/raid/can/nli_models/baseline/nli/outputs_bert_base_clark_1_seed23370/normal/
0.05786792577651799 0.07794352316021892
-0.022215955343460308 0.02703482338891347
0.0800838811199783 0.10496942344310056
factual score:
[0.6692333333333333, 0.6186666666666667, 0.6798] 0.6559 0.02667901103170109
TE:
[0.17496484716081254, 0.18611276726611947, 0.1704057694688245] 0.17716112796525216 0.0065977360753529096
TIE:
[0.06294446807694988, 0.07404294678696302, 0.05786792577651799] 0.06495178021347697 0.006754248178352705
[0.6668, 0.6174, 0.6777333333333333] 0.6539777777777777 0.026246711728199593
NIE:
[0.086922

In [5]:
df_bert = pd.read_json('/raid/can/nli_models/poe3/nli/outputs_poe_bert_base_korn_clark_1_seed13370/'+'raw_mm.jsonl', lines=True)

NameError: name 'pd' is not defined

In [ ]:
!ls /raid/can/nli_models/

In [ ]:
!ls -ltr /raid/can/nli_models/baseline/nli/outputs_bert_base_clark_1_seed13370/normal/


In [22]:
import pandas as pd

In [23]:
  df_bias_model = pd.read_json(data_path+'test'+'_prob_korn_lr_overlapping_sample_weight_3class.jsonl', lines=True)

In [34]:
if '-' in df_bias_model['gold_label'].value_counts():
    print(df_bias_model['gold_label'].value_counts()['-'])

168


In [9]:
df_bias_model

NameError: name 'df_bias_model' is not defined

In [124]:
import torch

In [125]:
x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

In [126]:
print('Gradient function for z =', z.grad_fn)
print('Gradient function for loss =', loss.grad_fn)

Gradient function for z = <AddBackward0 object at 0x7fa37bdb2490>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x7fa37bdb26a0>


In [127]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.1463, 0.1088, 0.0836],
        [0.1463, 0.1088, 0.0836],
        [0.1463, 0.1088, 0.0836],
        [0.1463, 0.1088, 0.0836],
        [0.1463, 0.1088, 0.0836]])
tensor([0.1463, 0.1088, 0.0836])


NameError: name 'model' is not defined

In [131]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

In [166]:
import os
import pandas as pd
from torchvision.io import read_image
import torch.nn as nn

class CustomImageDataset(Dataset):
    def __init__(self, probs,target_probs):
        self.probs  = torch.tensor(probs)
        self.target_probs = torch.tensor(target_probs)
    def __len__(self):
        return len(self.probs)

    def __getitem__(self, idx):
        return self.probs[idx], self.target_probs[idx]


In [169]:
df_bert['probs'][0]

[0.13495792448520602, 0.7968678474426271, 0.068174198269844]

In [170]:
train=CustomImageDataset(df_bert['probs'],df_bert['probs'])

In [171]:
class TempScale(nn.Module):
    def __init__(self):
        super(TempScale, self).__init__()
        self.T = nn.Parameter(torch.tensor(1.0))

    def forward(self, x):
        x = (x/self.T)/torch.sum(x/self.T)
        return x
model = TempScale()

In [172]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = dataloader.__len__()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [173]:
learning_rate = 1e-3
batch_size = 64
epochs = 5
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
train_loop(train, model, loss_fn, optimizer)

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [174]:
train.__len__()

30000

In [165]:
torch.tensor(df_bert['probs']).shape

torch.Size([30000, 3])